# PROYECTO FINAL - modificación de columnas

In [107]:
# Tratamiento de datos
import numpy as np
import pandas as pd
#import sidetable as stb

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import sidetable as stb

# Asunciones y Preprocesamiento
from scipy import stats
import math
from scipy.stats import levene
#import researchpy as rp
from sklearn.preprocessing import StandardScaler
import itertools
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols
import researchpy as rp # chi square

#Linear Regresion con Sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = False

In [108]:
df_bike = pd.read_csv("./bikes_visualizacion_parte_1.csv", index_col=0)
df_bike.sample(6)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
694,695,26-11-2019,winter,1,11,0,1,1,1,12.846653,16.95020,53.5417,3.125550,337,4750,5087
284,285,12-10-2018,winter,0,10,0,3,1,3,22.276653,25.88585,90.6250,16.626050,217,2199,2416
716,717,18-12-2019,winter,1,12,0,2,1,1,16.844153,20.48540,66.6250,14.834068,433,5124,5557
560,561,15-07-2019,autumn,1,7,0,0,0,1,30.579153,35.25980,71.7917,11.166689,1920,4111,6031
650,651,13-10-2019,winter,1,10,0,6,0,1,16.126653,19.56980,49.4583,9.791514,2252,4857,7109
427,428,04-03-2019,spring,1,3,0,0,0,1,13.359153,15.15105,40.3333,22.416257,710,2713,3423


# MODIFICACIONES EN NUESTRAS COLUMNAS:

# 1. Fecha:
- Tenemos "yr" (año) y "mnth" (mes) y obtenemos "day" a través de la separación de "dteday", que posteriormente eliminaremos:

In [109]:
df_bike["day"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[0]

In [110]:
df_bike=df_bike.reindex(columns=['instant', 'dteday', 'season', 'yr', 'mnth','day', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'])

In [111]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,01,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,02,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,03,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,04,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,05,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


------

# 2. Columna "season"
- Corregir de acuerdo con las fechas.

In [112]:
df_bike['dteday'] = df_bike['dteday'].apply(pd.to_datetime)

In [113]:
def meses(x):
    a = x.month

    if  a >= 1 and a<=3:
        return 'winter'
    elif a >= 4 and a<=6:
        return 'spring'
    elif a >= 7 and a<=9:
        return 'summer'
    else:
        return 'autumn'

In [114]:
df_bike['season'] = df_bike['dteday'].apply(meses)

In [115]:
df_bike.sample(5)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
99,100,2018-10-04,autumn,0,4,10,0,0,0,2,17.493347,21.33685,85.7500,9.833389,1188,1707,2895
339,340,2018-06-12,spring,0,12,06,0,2,1,3,18.962500,22.82000,94.9583,15.583061,126,2468,2594
619,620,2019-12-09,autumn,1,9,12,0,3,1,1,24.565847,28.50375,57.7083,8.833682,1050,6820,7870
152,153,2018-02-06,winter,0,6,02,0,4,1,1,29.315000,32.19710,30.5000,19.583229,736,4232,4968
267,268,2018-09-25,summer,0,9,25,0,0,0,2,26.000847,28.63185,84.5000,3.375406,1544,3466,5010


 - Encoding numérico para la columna season

In [116]:
scale_mapper = {"winter":1, "spring":3, "summer":4, 'autumn':2}

df_bike['season_encoding'] = df_bike["season"].replace(scale_mapper)

--------

# 8. "weathersit": 
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

In [117]:
scale_mapper_1 = {"1":3, "2":2, "3":1, '4':1}

In [118]:
df_bike['weathersit'] = df_bike["weathersit"].replace(scale_mapper_1)

---

# 9. Temperatura y Humedad

### Hay que hacer encoding con etiquetas para las graficas

In [119]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,1


In [120]:
df_bike[['temp','atemp','hum']] = df_bike[['temp','atemp','hum']].round(1)

###  Codificando variables

In [121]:
df_bike["temp_interval"] = pd.cut(df_bike["temp"], 5,labels = ["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [122]:
df_bike["temp_interval_num"] = pd.cut(df_bike["temp"], 5, ordered=True)

In [123]:
df_bike["hum_interval"] = pd.cut(df_bike["hum"], 5,labels = ["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [124]:
df_bike["hum_interval_num"] = pd.cut(df_bike["hum"], 5, ordered=True)

In [125]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]"
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]"
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]"
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,3,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]"
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,3,low,"(8.98, 15.56]",medium,"(38.88, 58.32]"


### Voy a hacer un ordinal encoding

In [126]:
from sklearn.preprocessing import OrdinalEncoder
orden = ["very_low", "low", "medium", "high", "very_high"] # Ordena

In [127]:
def ordinal_encoder(df, columna, orden_valores):
    
    # nos creamos un diccionario vacío para hacer el map después. 
    ordinal_dict = {}
    
    # iteramos por nuestra lista de valores usando el enumerate que recordamos nos devolvía también el índice o posición de cada elemento
    for i, valor in enumerate(orden_valores):
        ordinal_dict[valor]=i+1 # le sumamos uno a la posición para no tener valores de 0. 
    
    # aplicamos el map
    df[columna+"_ord"] = df[columna].map(ordinal_dict)
    return df

In [128]:
ordinal_encoder(df_bike, "temp_interval", orden)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]",1
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,3,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]",1
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,3,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,2019-12-27,autumn,1,12,27,0,4,1,2,10.4,11.3,65.3,23.458911,247,1867,2114,2,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
726,727,2019-12-28,autumn,1,12,28,0,5,1,2,10.4,12.8,59.0,10.416557,644,2451,3095,2,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
727,728,2019-12-29,autumn,1,12,29,0,6,0,2,10.4,12.1,75.3,8.333661,159,1182,1341,2,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2
728,729,2019-12-30,autumn,1,12,30,0,0,0,1,10.5,11.6,48.3,23.500518,364,1432,1796,2,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2


    Humedad

In [129]:
get_dummies= pd.get_dummies(df_bike["hum_interval"], prefix="hum",prefix_sep="_")

In [130]:
df_bike = pd.concat([df_bike, get_dummies], axis=1)

In [131]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord,hum_very_low,hum_low,hum_medium,hum_high,hum_very_high
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2,0,0,0,0,1
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,0,0,0,1,0
2,3,2018-03-01,winter,0,1,03,0,1,1,1,8.1,9.5,43.7,16.636703,120,1229,1349,1,very_low,"(2.367, 8.98]",medium,"(38.88, 58.32]",1,0,0,1,0,0
3,4,2018-04-01,spring,0,1,04,0,2,1,1,8.2,10.6,59.0,10.739832,108,1454,1562,3,very_low,"(2.367, 8.98]",high,"(58.32, 77.76]",1,0,0,0,1,0
4,5,2018-05-01,spring,0,1,05,0,3,1,1,9.3,11.5,43.7,12.522300,82,1518,1600,3,low,"(8.98, 15.56]",medium,"(38.88, 58.32]",2,0,0,1,0,0


---

# 10. "Windspeed": encoding

### Se ponen etiquetas a viento

In [132]:
df_bike['wind_interval'] = pd.cut(df_bike['windspeed'], 5, labels=["very_low", "low", "medium", "high", "very_high"], ordered=True)

In [133]:
df_bike['wind_interval'] = df_bike['wind_interval'].astype('object')

### Encoding numérico para la columna wind

In [134]:
scale_mapper = {"very_low":4, "low":3, "medium":2, 'high':1, 'very_high':1}

df_bike['wind_num_encoding'] = df_bike["wind_interval"].replace(scale_mapper)

In [135]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,season_encoding,temp_interval,temp_interval_num,hum_interval,hum_interval_num,temp_interval_ord,hum_very_low,hum_low,hum_medium,hum_high,hum_very_high,wind_interval,wind_num_encoding
0,1,2018-01-01,winter,0,1,01,0,6,0,2,14.1,18.2,80.6,10.749882,331,654,985,1,low,"(8.98, 15.56]",very_high,"(77.76, 97.2]",2,0,0,0,0,1,low,3
1,2,2018-02-01,winter,0,1,02,0,0,0,2,14.9,17.7,69.6,16.652113,131,670,801,1,low,"(8.98, 15.56]",high,"(58.32, 77.76]",2,0,0,0,1,0,medium,2


In [136]:
df_bike.to_csv('bike_columnas_modificadas.csv')